In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math

# 주성분분석(Principal Component Analysis)을 하기 위한 라이브러리
from sklearn.decomposition import PCA

# 파일 경로 확인용

import os
import glob

# .py 파일 가져오기
# import (폴더이름.)파일이름
import src.facenet as facenet
import src.align.detect_face as detect_face

# jupyter notebook 펭지에 그리기
%matplotlib inline

# 학습되어 있는 모델을로드하는 함수
# 입력 : protobuf 파일(모델이 정의되어 있음)
# 리턴 :
#     - single_image : 영상을 입력하는 placeholder (session, run feed_dict 에 사용)
#     - embeddings : 네트어크의 출력 값, 512차원의 추출된 얼굴 특징 벡터

def load_model(pb_path, image_size=(160,160)) :
    tf.reset_default_graph()
    
    single_image = tf.placeholder(tf.int32, (None, None, 3))
    float_image = tf.cast(single_image, tf.float32)
    float_image = float_image/255
    batch_image = tf.expand_dims(float_image, 0)
    resized_image = tf.image.resize(batch_image, image_size)
    
    phase_train = tf.placeholder_with_default(False, shape=[])
    
    input_map = {'image_batch' : resized_image, 'phase_train' : phase_train}
    model = facenet.load_model(pb_path, input_map)
    
    embeddings = tf.get_default_graph().get_tensor_by_name("embeddings: 0")
    
    return single_image, embeddings

# 영상 경로를 입력받아 로드하고, return해주는 함수
def load_image(image_path) :
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    return image

# 두 벡터간의 거리를 계산하는 함수

def calc_distance(embedding1, embedding2) :
    # Euclidian distance
    diff = np.subtract(embedding1, embedding2)
    dist = np.sum(np.square(diff), 0)
    
    return dist


In [2]:
#############과제##############
# 사용함수 
# calc_distance (두 벡터간의 거리 계산 함수. 서로 다른 두 얼굴의 특징벡터간 거리.)
# detect_face.creat_mtcnn ()
# load_model

tf.reset_default_graph()
 
single_image, embeddings = load_model("./models/20180402-114759.pb")

sess = tf.Session()
pnet, rnet, onet = detect_face.create_mtcnn(sess,None)#학습되어 있는 detection 모델 로드


################################


#내 얼굴의 특징백터 (embedding) 추출
path_me = glob.glob("./data/faces/chaelin/*")
embed_me = []

for path in path_me :
    img = load_image(path)
    result = sess.run(embeddings, feed_dict={single_image : img})
    result = result[0]
    embed_me.append(result)

embed_me = np.array(embed_me)
print(embed_me.shape)

# 첫 번째 얼굴 사진의 특징벡터 확인
print(embed_me[0])

Model filename: ./models/20180402-114759.pb
Instructions for updating:
Use tf.gfile.GFile.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
(24, 512)
[ 9.44873840e-02  2.47697737e-02 -3.28789023e-03  5.33525161e-02
  2.19947174e-02 -2.26591974e-02 -4.75082658e-02  2.05385331e-02
 -1.40227228e-02 -4.23607565e-02  4.05533537e-02  2.08814139e-03
  4.13690973e-03  2.73972321e-02  1.75733101e-02 -9.56316069e-02
  2.53071031e-03  5.79846650e-03  4.57997955e-02  4.97211926e-02
  9.41700488e-03 -2.47731134e-02  5.26755117e-02 -1.01825364e-01
  3.96296661e-03 -5.24287811e-03  8.91560167e-02 -2.11817585e-02
  5.38082756e-02 -1.29293082e-02  5.10510430e-02  2.86690407e-02
 -5.19351549e-02  1.75030529e-02 -6.85283123e-03 -6.71764314e-02
 -6.02130331e-02 -1.64456461e-02 -4.29300256e-02  3.57171372e-02
 -8.00881386e-02  7.71067431e-03 -1.68482668e-03  3.72561254e-02
 -7.24110240e-03 -3.58745754e-02  

In [15]:
# 단일 RGB영상과 pre-trained 네트워크(P/R/O net)를 입력받아
# 1. face detection
# 2. crop(margin추가)
# 3. resize 하고
# 검출된 얼굴 영상들의 리스트를 반환하는 함수

def crop_faces(image, pnet, rnet, onet) :
    # face detection 관련 파라미터
    minsize = 10 # minimum size of face
    threshold = [0.6, 0.7, 0.7] # thress step's threshold
    factor = 0.709 # scale factor
    ##################################################
    
    # crop 관련 파라미터
    margin = 44 # 상하좌우 여백
    image_size = 160 # resize(scaling) 크기
    
    h,w,_ = np.shape(image)
    
    # 얼굴 검출
    bounding_boxes, points = detect_face.detect_face(image, minsize, pnet, rnet, onet, threshold, factor)
    # bounding_boxes : 검출된 사각형 영역, [x1, y1, x2, y2, 확률]로 이루어진 리스트
    # points : 검출된 얼굴의 주요 landmark [x1, x2, x3, x4, x5, y1, y2, y3, y4, y5]로 이루어진 리스트
    
    faces = []
    for box in bounding_boxes :
        box = np.int32(box)
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(box[0] - margin/2, 0)
        bb[1] = np.maximum(box[1] - margin/2, 0)
        bb[2] = np.maximum(box[2] - margin/2, w)
        bb[3] = np.maximum(box[3] - margin/2, h)
        cropped = image[bb[1]:bb[3], bb[0]:bb[2], :]
        scaled = cv2.resize(cropped, (image_size, image_size), interpolation=cv2.INTER_LINEAR)
        
        faces.append(scaled)
        
    return faces, bounding_boxes

In [18]:
#얼굴인식(캠)

import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, frame = cap.read()

    frame = cv2.resize(frame,(400,225))
#     detected_faces = FaceDetection(frame)
    faces, bounding_boxes = crop_faces(frame, pnet, rnet, onet)
    
#     cv2.imshow("frame", frame)
#     cv2.imshow("detected", frame)
    
    squared_img = frame.copy()
    for i in range(len(bounding_boxes)):
        box=bounding_boxes[i]
        face=faces[i]
        box = np.int32(box)
        p1 = (box[0], box[1])
        p2 = (box[2], box[3])

        result_x = sess.run(embeddings, feed_dict={single_image : face})
        result_x = result_x[0]
        distance_th = 1 # 거리가 얼마 미만이어야 나로 분류할지
        distance1 = calc_distance(embed_me[0], result_x)
        distance2 = calc_distance(embed_me[3], result_x)
        avg_distance = (distance1 + distance2)/2
#         print("distance:", avg_distance)
        if(avg_distance < distance_th):
            cv2.rectangle(squared_img, p1, p2, color=(0,255,0))
            cv2.imshow(' ', squared_img)
        else :
            cv2.rectangle(squared_img, p1, p2, color=(0,0,255))
            cv2.imshow(' ', squared_img)
#         cv2.rectangle(squared_img, p1, p2, color=(255,0,0))
        print(avg_distance)
        
        
#     cv2.imshow(' ', squared_img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

for i in range(1,5):
    cv2.waitkey(1)

0.7727456092834473
0.7744195461273193
0.736672043800354
0.7818558216094971
0.7843027710914612
0.7465827465057373
0.946595311164856
0.7650546431541443
0.8077779412269592
1.0760785341262817
1.0860120058059692
1.1063001155853271
1.0984828472137451
1.0159811973571777
0.958532989025116
0.8875497579574585
1.093224048614502
1.161869764328003
1.2360005378723145
1.1676740646362305
1.1246774196624756
1.277126431465149
1.2403697967529297
1.2891650199890137
1.2960609197616577
1.2547448873519897
1.1392152309417725
1.1748408079147339
1.150874137878418
1.0453014373779297
1.0566798448562622
1.1118221282958984
1.1016559600830078
1.117511510848999
1.1021475791931152
1.1595921516418457
1.0857245922088623
1.1091541051864624
1.1553782224655151
1.097258448600769
1.11787748336792
1.0678362846374512
1.0785207748413086
1.079156756401062
1.0197710990905762
1.143244743347168
1.1219277381896973
1.051998257637024
1.0412933826446533
1.0211718082427979
0.9951075911521912
1.002004623413086
0.9544651508331299
0.971219

KeyboardInterrupt: 